In [1]:
import pandas as pd
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import defaultdict
import joblib

import sys

import os

import time
start_time = time.time()


infile='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/Randomforest/developing/realtrainingtable/BL14_genepromSM_0.7.txt_REALTRAINbestref.txt_result_dupindex_binnedstats.pkl_totalformatteddata.pkl'

outfol=infile+"_model"

os.mkdir(outfol)

gtcol='label2'

n_trees=10000



indf=pd.read_pickle(infile)

indf.head()

,index,CD4-others,CD8-others,nB-others,NK-others,Mn-others,mNeu-others,m8-others,DC-others,Eo-others,...,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,label,Last,label2,deltaMedian,deltaMean
0,CD4.HWI-7001432L_73:2:1104:13757:60166#13,0.730141,0.463423,-0.136423,-0.109141,-0.114885,-0.120269,-0.070372,-0.112731,-0.125295,...,0.730141,CD4-others,0.730141,CD4-others,CD4_fivemixgeneproTR_NR_767146_insilmix5_sorte...,CD4,HWI-7001432L_73:2:1104:13757:60166#13,CD4,-0.110936,7.930164e-17
1,CD4.HWI-7001432L_73:2:1106:7694:42940#11,1.517577,1.277064,-0.452474,-0.357346,-0.437756,-0.376372,0.054756,-0.386423,-0.443859,...,1.517577,CD4-others,1.517577,CD4-others,CD4_fivemixgeneproTR_NR_767146_insilmix5_sorte...,CD4,HWI-7001432L_73:2:1106:7694:42940#11,CD4,-0.338321,3.965082e-17
2,CD4.HWI-7001432L_73:2:1109:10401:26969#13,0.736051,0.507026,-0.147385,0.289487,-0.294923,-0.036103,-0.201769,-0.306769,-0.279846,...,0.736051,CD4-others,0.736051,CD4-others,CD4_fivemixgeneproTR_NR_767146_insilmix5_sorte...,CD4,HWI-7001432L_73:2:1109:10401:26969#13,CD4,-0.109154,9.417070e-17
3,CD4.HWI-7001432L_73:2:1208:13146:22328#11,2.309282,2.030359,-0.890974,-0.437590,-0.843949,-0.859744,0.232436,-0.801590,-0.810564,...,2.309282,CD4-others,2.309282,CD4-others,CD4_fivemixgeneproTR_NR_767146_insilmix5_sorte...,CD4,HWI-7001432L_73:2:1208:13146:22328#11,CD4,-0.753487,1.268826e-16
4,CD4.HWI-7001432L_73:2:1208:16168:38496#13,1.494692,1.210385,-0.419000,-0.154795,-0.370538,-0.410385,-0.136487,-0.414333,-0.348641,...,1.494692,CD4-others,1.494692,CD4-others,CD4_fivemixgeneproTR_NR_767146_insilmix5_sorte...,CD4,HWI-7001432L_73:2:1208:16168:38496#13,CD4,-0.304218,7.930164e-18


In [2]:
features=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others','LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg','deltaMedian', 'deltaMean']

In [3]:
scorecolumns=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others']

In [4]:
for score in scorecolumns:
    
    tempct=score.replace('-others','')
    
 
  
    indfsingleCT=indf[(indf[gtcol]==tempct) | (indf[gtcol]=='NOT'+tempct)].copy()
    
    
    
    indfsingleCT.loc[indfsingleCT[gtcol]==tempct,gtcol]=1
    indfsingleCT.loc[indfsingleCT[gtcol]=='NOT'+tempct,gtcol]=0
    

    
    indfsingleCT[gtcol]=indfsingleCT[gtcol].astype(int)
    
    
    trainY=indfsingleCT[gtcol]
    trainX=indfsingleCT[features]
    
    
    
    
   
    
    
    RDF_classifier = RandomForestClassifier(oob_score=True,n_estimators=n_trees)
    
    RDF_classifier.fit(trainX, trainY)
    
    joblib.dump(RDF_classifier, outfol+"/"+tempct+".pkl")

In [5]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

259.4489059448242
